In [ ]:
!git clone https://github.com/princeton-nlp/SimCSE.git

In [ ]:
%cd SimCSE

In [ ]:
!bash data/download_wiki.sh

In [ ]:
!bash /kaggle/working/SimCSE/data/download_nli.sh

In [ ]:
!mv nli_for_simcse.csv data/nli_for_simcse.csv

# Kiểm tra lại để chắc chắn file đã ở đúng chỗ
!ls -l data/nli_for_simcse.csv

In [ ]:
!bash SentEval/data/downstream/download_dataset.sh

In [ ]:
!python setup.py develop

In [ ]:
!pip install -r requirements.txt

In [ ]:
!conda install -c conda-forge libstdcxx-ng -y

In [ ]:
import shutil
import os

# Danh sách các thư mục dữ liệu vừa giải nén ra ở root
folders_to_move = ['CR', 'MPQA', 'MR', 'MRPC', 'SICK', 'SNLI', 'SST', 'STS', 'SUBJ', 'TREC']
target_dir = 'SentEval/data/downstream/'

# Tạo thư mục đích nếu chưa có
os.makedirs(target_dir, exist_ok=True)

print(f"Đang di chuyển dữ liệu vào {target_dir}...")
for folder in folders_to_move:
    if os.path.exists(folder):
        # Nếu thư mục đích đã có folder này thì xóa đi để ghi đè (tránh lỗi)
        dest_path = os.path.join(target_dir, folder)
        if os.path.exists(dest_path):
            shutil.rmtree(dest_path)
        
        shutil.move(folder, target_dir)
        print(f" -> Đã chuyển: {folder}")
    else:
        print(f" -> Không tìm thấy {folder} ở root (có thể đã chuyển rồi)")

print("Hoàn tất di chuyển dữ liệu!")

In [ ]:
# 1. (Tùy chọn) Kiểm tra xem file thư viện mới có tồn tại không
!ls -l /opt/conda/lib/libstdc++.so.6

!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/conda/lib && python train.py \
    --model_name_or_path bert-base-uncased \
    --train_file data/nli_for_simcse.csv \
    --output_dir result/my-sup-simcse-bert-base-uncased \
    --num_train_epochs 3 \
    --per_device_train_batch_size 64 \
    --learning_rate 5e-5 \
    --max_seq_length 32 \
    --evaluation_strategy steps \
    --metric_for_best_model stsb_spearman \
    --load_best_model_at_end \
    --eval_steps 125 \
    --pooler_type cls \
    --overwrite_output_dir \
    --temp 0.05 \
    --do_train \
    --do_eval \
    --fp16